In [1]:
%pip install -r requirements.txt -Uq

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import the class modules that support the Text to SQL pipeline
import app_constants as app_consts
import identify_service_facade
import llm_facade
import rdbms_facade

import pre_process_request
import prepare_request
import test_cases

In [3]:
# Instantiate objects for the classes that will be used
test_user_requests = test_cases.TestCases()
llm_service_facade = llm_facade.LlmFacade(app_consts.BEDROCK_MODEL_ID)
id_service_facade = identify_service_facade.IdentityServiceFacade()
rdbms_service_facade = rdbms_facade.RdbmsFacade()

request_pre_processor = pre_process_request.PreProcessRequest(llm_service_facade)
request_preparer = prepare_request.PrepareRequest()

### Pull an example user request from the test set

In [4]:
user_request = test_user_requests.get_test_case(4)
user_request

'In what games did Isabelle Werth, Nedo Nadi and Allyson Felix compete?'

### Step 1 - Pre-process the use request, determine their intent, and if there are identifiers that need to be mapped to IDs

In [5]:
pre_processed_request = request_pre_processor.run(user_request)
intent = pre_processed_request[app_consts.INTENT]
pre_processed_request

{'user_query': 'In what games did Isabelle Werth, Nedo Nadi and Allyson Felix compete?',
 'intent': 'olympics',
 'named_resources': {'allyson felix', 'isabelle werth', 'nedo nadi'}}

### Step 2 - Map identifiers/references to database IDs

In [6]:
named_resources = pre_processed_request[app_consts.NAMED_RESOURCES]
if len(named_resources) > 0:
    identifiers = id_service_facade.resolve(named_resources)
    # add identifies to the pre_processed_request object
    pre_processed_request[app_consts.IDENTIFIERS] = identifiers
else:
    pre_processed_request[app_consts.IDENTIFIERS] = []
pre_processed_request

{'user_query': 'In what games did Isabelle Werth, Nedo Nadi and Allyson Felix compete?',
 'intent': 'olympics',
 'named_resources': {'allyson felix', 'isabelle werth', 'nedo nadi'},
 'identifiers': [{'id': 34551, 'role': 32, 'name': 'allyson felix'},
  {'id': 129726, 'role': 32, 'name': 'isabelle werth'},
  {'id': 84026, 'role': 32, 'name': 'nedo nadi'}]}

### Step 3 - Prepare the prompt for LLM and generate SQL preamable if neccessary

In [7]:
prepared_request = request_preparer.run(pre_processed_request)
prepared_request

{'llm_prompt': 'You are a SQL expert. Given the following SQL tables definitions, generate SQL language to answer the user\'s question. \n\nEach user question is about the Olympic Games.\nProduce SQL ready for use with a SQLITE database.\nOutput the result in a JSON format with one key "sql".\nAnswer the question immediately without preamble.\n*important* Strictly follow the rules in the <rule> tags for generating the SQL \n\n\n<rule>\nThe athletes_in_focus is a temporary table that has been pre-populated with the specific set of \'id\' values of the athletes applicable for this query.\n</rule>\n<rule>\nNo filters need to be applied to the athletes_in_focus to get the correct set of athlete identifiers for queries  about specific athletes. \n</rule>\n<rule> \nEven when the user\'s question is about specific athletes, these athletes, and only these athletes, are in the athletes_in_focus table\n</rule>\n<rule>\nOnly use tables and columns explicitly described with the <SQL> tags\n</rule>

### Step 4 - Call the LLM to generate the SQL

In [8]:
llm_response = llm_service_facade.invoke(prepared_request[app_consts.LLM_PROMPT])
generated_sql = llm_response[app_consts.LLM_OUTPUT]
generated_sql

{'sql': 'SELECT g.games_name, g.games_year FROM athletes_in_focus a JOIN games_competitor gc ON gc.person_id = a.id JOIN games g ON gc.games_id = g.id;'}

### Step 5 - Run the generated SQL against the RDBMS

In [9]:
sql_script = prepared_request[app_consts.SQL_PREAMBLE] + [generated_sql[app_consts.SQL]]
database = app_consts.get_database_for_intent(intent)
results = rdbms_service_facade.execute_sql(database, sql_script)
results

{'rdbms_output': [('games_name', 'games_year'),
  ('2004 Summer', 2004),
  ('2008 Summer', 2008),
  ('2012 Summer', 2012),
  ('2016 Summer', 2016),
  ('1912 Summer', 1912),
  ('1920 Summer', 1920),
  ('1992 Summer', 1992),
  ('1996 Summer', 1996),
  ('2000 Summer', 2000),
  ('2008 Summer', 2008),
  ('2016 Summer', 2016)],
 'processing_status': 'success'}